In [1]:
#import pathlib
import json
import nzshm_model as nm
import solvis

from solvis.filter import FilterRuptureIds, FilterParentFaultIds, FilterSubsectionIds

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")

In [138]:
TARGET_FAULTS = ['Masterton', 'Ohariu']
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)
list(ruptures)

[305289,
 287882,
 287883,
 307466,
 398713,
 287886,
 305295,
 303633,
 303637,
 396573,
 303645,
 305956,
 303144,
 287935,
 304457,
 273738,
 365792,
 304481,
 332519,
 93817]

In [139]:
surfaces_all = solution.fault_surfaces()
subsections = FilterSubsectionIds(solution).for_rupture_ids(ruptures)
surfaces = surfaces_all[surfaces_all["FaultID"].isin(subsections)]
surfaces.shape

(87, 14)

In [140]:
from IPython.display import GeoJSON
GeoJSON(json.loads(surfaces.to_json()))

<IPython.display.GeoJSON object>

In [142]:
## new solution
%time new_solution = solution.filter_solution(solution, rupture_ids=ruptures)

CPU times: user 12.2 s, sys: 227 ms, total: 12.5 s
Wall time: 12.5 s


In [143]:
new_surfaces_all = new_solution.fault_surfaces()

new_ruptures = FilterRuptureIds(new_solution)\
    .for_parent_fault_names(TARGET_FAULTS)

assert list(new_ruptures) == list(ruptures)

new_subsections = FilterSubsectionIds(new_solution).for_rupture_ids(new_ruptures)
new_surfaces = new_surfaces_all[new_surfaces_all["FaultID"].isin(new_subsections)]
new_surfaces.shape

(87, 14)

In [144]:
GeoJSON(json.loads(new_surfaces.to_json()))

<IPython.display.GeoJSON object>

In [145]:
## check new participation rates are smaller

rates = solution.section_participation_rates()
new_rates = new_solution.section_participation_rates()

print(f'solution rates {rates.shape}')
print(f'new_solution rates {new_rates.shape}')

assert new_rates[new_rates.index==132].participation_rate.sum() < rates[rates.index==132].participation_rate.sum()
assert new_rates.participation_rate.sum() < rates.participation_rate.sum()

solution rates (2325, 2)
new_solution rates (87, 2)


In [88]:
new_solution.to_archive(
    archive_path="play_solution_compat.zip",
    base_archive_path="NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip",
    compat=True)

/opt/conda/lib/python3.11/zipfile.py:1566: UserWarning: Duplicate name: 'WARNING.md'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [90]:
cc_solution_compat = solvis.InversionSolution.from_archive("play_solution_compat.zip")
cc_solution = solvis.InversionSolution.from_archive("play_solution.zip")

In [91]:
cc_rates = cc_solution.section_participation_rates() # no rates here!! 
cc_rates_compat = cc_solution_compat.section_participation_rates()

In [93]:
cc_rates_compat

,Rupture Index,participation_rate
section,,
132,0,0.000006
133,0,0.000006
134,0,0.000006
135,0,0.000006
136,0,0.000006
...,...,...
2295,16,0.0
2296,35,0.0
2297,35,0.0


In [94]:
assert new_rates.participation_rate.sum() == cc_rates_compat.participation_rate.sum()

In [96]:
cc_surfaces_all = cc_solution_compat.fault_surfaces()

ruptures = FilterRuptureIds(cc_solution_compat)\
    .for_parent_fault_names(TARGET_FAULTS)
# list(new_ruptures)

# assert list(new_ruptures) == list(ruptures)

subsections = FilterSubsectionIds(cc_solution_compat).for_rupture_ids(ruptures)
surfaces = cc_surfaces_all[cc_surfaces_all["FaultID"].isin(subsections)]
surfaces.shape

(87, 14)

In [97]:
GeoJSON(json.loads(surfaces.to_json()))

<IPython.display.GeoJSON object>

## more sanity checks

In [ ]:
# fault_sections
assert cc_solution_compat.fault_sections.shape == new_solution.fault_sections.shape
assert cc_solution_compat.fault_sections.columns.all() == new_solution.fault_sections.columns.all()
assert cc_solution_compat.fault_sections.index.all() == new_solution.fault_sections.index.all()


In [132]:
# ruptures
assert cc_solution_compat.ruptures.shape == new_solution.ruptures.shape
assert cc_solution_compat.ruptures.columns.all() == new_solution.ruptures.columns.all()

# rupture indexes are different due to the compatible
# assert cc_solution.ruptures.index.all() == new_solution.ruptures.index.all()
# assert cc_solution_compat.ruptures.index.all() == new_solution.ruptures.index.all()

print(cc_solution.ruptures.index.tolist())
print(cc_solution_compat.ruptures.index.tolist())
print(new_solution.ruptures.index.tolist())

cc_solution_compat.ruptures.tail()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[93817, 273738, 287882, 287883, 287886, 287935, 303144, 303633, 303637, 303645, 304457, 304481, 305289, 305295, 305956, 307466, 332519, 365792, 396573, 398713]


,Rupture Index,Magnitude,Average Rake (degrees),Area (m^2),Length (m)
15,15,7.689291,173.514908,3085278208.0,149419.578125
16,16,8.092493,170.060974,7807235072.0,344133.71875
17,17,7.687795,172.046402,3074670080.0,148691.578125
18,18,7.676656,174.574341,2996815872.0,148867.984375
19,19,8.096851,170.340118,7885977600.0,347838.75


In [131]:
# rupture_sections
assert cc_solution_compat.rupture_sections.shape == new_solution.rupture_sections.shape
assert cc_solution_compat.rupture_sections.columns.all() == new_solution.rupture_sections.columns.all()
assert cc_solution_compat.rupture_sections.index.all() == new_solution.rupture_sections.index.all()
assert cc_solution_compat.rupture_sections.section.max() == new_solution.rupture_sections.section.max()

cc_solution_compat.rupture_sections.tail()

,rupture,section
299,19,1698
300,19,1697
301,19,1696
302,19,2297
303,19,2296


In [134]:
# rupture_sections_with_rupture_rates
assert cc_solution_compat.rs_with_rupture_rates.shape == new_solution.rs_with_rupture_rates.shape
assert cc_solution_compat.rs_with_rupture_rates.columns.all() == new_solution.rs_with_rupture_rates.columns.all()

# data columns are OK
assert cc_solution_compat.rs_with_rupture_rates['Annual Rate'].sum() == new_solution.rs_with_rupture_rates['Annual Rate'].sum()
assert cc_solution_compat.rs_with_rupture_rates.section.all() == new_solution.rs_with_rupture_rates.section.all()

# assert cc_solution_compat.rs_with_rupture_rates.index.all() == new_solution.rs_with_rupture_rates.index.all()
print()
cc_solution_compat.rs_with_rupture_rates.tail()
# new_solution.rs_with_rupture_rates

,key_0,Rupture Index,Annual Rate,Magnitude,Average Rake (degrees),Area (m^2),Length (m),section
19,19,19,0.0,8.096851,170.340118,7885977600.0,347838.75,1698
19,19,19,0.0,8.096851,170.340118,7885977600.0,347838.75,1697
19,19,19,0.0,8.096851,170.340118,7885977600.0,347838.75,1696
19,19,19,0.0,8.096851,170.340118,7885977600.0,347838.75,2297
19,19,19,0.0,8.096851,170.340118,7885977600.0,347838.75,2296


In [135]:
new_solution.rs_with_rupture_rates.tail()

,key_0,Rupture Index,Annual Rate,Magnitude,Average Rake (degrees),Area (m^2),Length (m),section
398713,398713,398713,0.0,8.096851,170.340118,7885977600.0,347838.75,1698
398713,398713,398713,0.0,8.096851,170.340118,7885977600.0,347838.75,1697
398713,398713,398713,0.0,8.096851,170.340118,7885977600.0,347838.75,1696
398713,398713,398713,0.0,8.096851,170.340118,7885977600.0,347838.75,2297
398713,398713,398713,0.0,8.096851,170.340118,7885977600.0,347838.75,2296
